<a href="https://colab.research.google.com/github/aarush-umap/asl-alphabet-classification/blob/master/evaluate_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluate tflite model
This notebook serves as a pipeline to load and test a tflite model.

Must be tested on a lighter architecture (e.g. raspberry pi)

## Setup packages

In [1]:
!pip install tflite-runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.3 MB/s eta 0:00:00


In [2]:
import tflite_runtime.interpreter as tflite
import os
import sys
import cv2
import numpy as np

## Connect to Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'asl-alphabet'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
sys.path.append(GOOGLE_DRIVE_PATH)
TEST_DIR = os.path.join(GOOGLE_DRIVE_PATH, 'asl_alphabet_test', 'asl_alphabet_test')

['resnet.py', '__pycache__', 'best_weights', 'model.tflite', 'best_weights_no_LR', 'asl_alphabet_test', 'best_weights_resized', 'classification_google.ipynb', 'model_with_lr.tflite', 'evaluate_tflite.ipynb']


## Setup variables and methods

In [5]:
img_height = 200
img_width = 200
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

In [16]:
def predict_letter(model, img):
  print(f'calculating prediction')
  img_array = np.expand_dims(img, 0)

  input = model.get_input_details()[0]
  output = model.get_output_details()[0]
  img_typed = img_array.astype(input["dtype"])
  try:
    model.set_tensor(input['index'], img_typed)
  except:
    print(f'model input should be: {input["dtype"]} but was {img_typed.dtype}')
    return None

  model.invoke()

  prediction = model.get_tensor(output['index'])
  predicted_class = classes[np.argmax(prediction)]
  return predicted_class

In [17]:
def load_image(letter):
  path = TEST_DIR + f"/{letter}_test.jpg"
  img = cv2.imread(path)
  img = cv2.resize(img, (img_height, img_width))
  print(f'loaded {letter}_test.jpg of type: {img.dtype}')
  return img


In [18]:
def load_lite(path):
  print(f'Loading tflite model from {path}')
  model = tflite.Interpreter(path)
  model.allocate_tensors()
  return model


In [19]:
letter = 'A'
path_to_model = os.path.join(GOOGLE_DRIVE_PATH, 'model_with_lr.tflite')
net50lite = load_lite(path_to_model)
image = load_image(letter)
predicted = predict_letter(net50lite, image)

print(f'Model predicted: {predicted}, expecting {letter}')

Loading tflite model from drive/My Drive/asl-alphabet/model_with_lr.tflite
loaded A_test.jpg of type: uint8
calculating prediction
Model predicted: A, expecting A
